In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re

# Sumo Matches Forecast

https://data.world/cervus/sumo-japan

https://josh-sulkers.medium.com/building-a-sumo-wrestling-match-predictor-using-machine-learning-2ac95c5e20a3

first wrestler result after the bout (final result in brackets)





Wrestler performance can be viewed from multiple prospectives:
- the overall career wins 
- the wins in the current tournament; each tournament is 15 days, 1 bout / day
- bouts vs current opponent
This way we can capture the overall, current, and vs opponent performance of the rikishi.

We can use several metrics: 
- absolute number of wins
- proportion of wins
- weighted moving average (WMA) (e.g. last 4 bouts)
    - exponentially weighed
    - other (custom) weights

Absolute number. WMA recent observations are given relatively more weight than older ones.


**Features**

Wrestler characteristics
- age
- height
- weight
- body mass index - calculated

Wrestler performance
- wins vs current opponent, exponentially WMA of wins for the last 10 bouts
- wins vs 


career_wins1
tournament_wins1
enemy_wins


We will treat the problem as a 2-class-value classification problem:
- class 0 - wrestler1 loses (the opponent wins)
- class 1 - wrestler1 wins



Each bout is recorded twice, once from the prospective of each wrestler. 

## Prepare data

### Preprocess tournament data

In [123]:
# Read tournament records

results = pd.read_csv("data/results.csv")
results.columns = [
    'basho', 'day',
    'id1', 'rank1', 'name1', 'tournament_score1', 'bout_outcome1',
    'kimarite',
    'id2', 'rank2', 'name2', 'tournament_score2', 'bout_outcome2',
]
results.tail()

basho  day    id1 rank1         name1 tournament_score1  \
224775  2022.09   15  12370   S1e  Wakatakakage              11-4   
224776  2022.09   15  11980   M6e  Wakamotoharu              10-5   
224777  2022.09   15  12210   O2w     Mitakeumi              4-11   
224778  2022.09   15  12130   O1w        Shodai              4-11   
224779  2022.09   15  12191   O1e    Takakeisho              10-5   

        bout_outcome1   kimarite    id2 rank2         name2 tournament_score2  \
224775              1   yorikiri   2879   M5w     Sadanoumi               9-6   
224776              1   yorikiri  12210   O2w     Mitakeumi              4-11   
224777              0   yorikiri  11980   M6e  Wakamotoharu              10-5   
224778              0  oshidashi  12191   O1e    Takakeisho              10-5   
224779              1  oshidashi  12130   O1w        Shodai              4-11   

        bout_outcome2  
224775              0  
224776              0  
224777              1  
224778              1  
224779              0

In [124]:
# Filter out Makuuchi division

makuuchi_regex = r"([YOSKM]+[1-9]+)"
results.rank1 = results.rank1.str.extract(makuuchi_regex, expand=True)
results.rank2 = results.rank2.str.extract(makuuchi_regex, expand=True)

results = results.dropna().reset_index(drop=True)

In [5]:
# # Drop duplicate matches

# results.loc[results.rikishi1_id > results.rikishi2_id, "match_id"] = \
# results.basho.astype(str) + "," + results.day.astype(str) + "," + results.rikishi1_id.astype(str) + "," + results.rikishi2_id.astype(str)

# results.loc[results.rikishi1_id < results.rikishi2_id, "match_id"] = \
# results.basho.astype(str) + "," + results.day.astype(str) + "," + results.rikishi2_id.astype(str) + "," + results.rikishi1_id.astype(str)

# results = results.drop_duplicates(subset="match_id")
# results = results.drop(columns="match_id")
# results = results.reset_index(drop=True)

In [6]:
# Extract year and month columns

def get_year_and_month_from_date(date_column):
    year_and_month = date_column.astype(str).str.split(".", expand=True)
    year_and_month.columns = ["year", "month"]
    year_and_month = year_and_month.astype(int)
    
    return year_and_month

year_and_month = get_year_and_month_from_date(results.basho)
results = pd.concat([results, year_and_month], axis=1)
results = results.drop(columns="basho")

In [7]:
# Create columns for tournament wins and losses (before the current bout)

score_regex = r"([0-9]+-[0-9]+)"

# Wrestler1
wins_and_losses = results.tournament_score1.str.extract(score_regex)[0]
wins_and_losses = wins_and_losses.str.split("-", expand=True)
wins_and_losses.columns = ["tournament_wins1", "tournament_losses1"]
wins_and_losses = wins_and_losses.astype(int)
results = pd.concat([results, wins_and_losses], axis=1)

# Subtract the current bout outcome to get wins/losses correct count before the fight
results.tournament_wins1 -= results.bout_outcome1
results.tournament_losses1 -= results.bout_outcome2


# Repeat for wrestler2
wins_and_losses = results.tournament_score2.str.extract(score_regex)[0]
wins_and_losses = wins_and_losses.str.split("-", expand=True)
wins_and_losses.columns = ["tournament_wins2", "tournament_losses2"]
wins_and_losses = wins_and_losses.astype(int)
results = pd.concat([results, wins_and_losses], axis=1)
results.tournament_wins2 -= results.bout_outcome2
results.tournament_losses2 -= results.bout_outcome1

In [8]:
# Encode the wrestler rank

power_ranks = pd.read_csv("data/power_rank.csv")
power_ranks_dict = dict(zip(power_ranks["rank"], power_ranks["power"]))

results.rank1 = results.rank1.replace(power_ranks_dict)
results.rank2 = results.rank2.replace(power_ranks_dict)

In [9]:
# Drop irrelevant columns
results = results.drop(columns=["tournament_score1", "tournament_score2", "name1", "name2", "kimarite"])

In [10]:
# Re-order columns
results = results[[
    'year',
    'month',
    'day',
    'id1',
    'rank1',
    'bout_outcome1',
    'tournament_wins1',
    'tournament_losses1',
    'id2',
    'rank2',
    'bout_outcome2',
    'tournament_wins2',
    'tournament_losses2',
]]

In [11]:
# Print the clean tournament data
results.tail()

year  month  day    id1  rank1  bout_outcome1  tournament_wins1  \
133519  2022      9   15  12370     19              1                10   
133520  2022      9   15  11980     12              1                 9   
133521  2022      9   15  12210     20              0                 4   
133522  2022      9   15  12130     20              0                 4   
133523  2022      9   15  12191     20              1                 9   

        tournament_losses1    id2  rank2  bout_outcome2  tournament_wins2  \
133519                   4   2879     13              0                 9   
133520                   5  12210     20              0                 4   
133521                  10  11980     12              1                 9   
133522                  10  12191     20              1                 9   
133523                   5  12130     20              0                 4   

        tournament_losses2  
133519                   5  
133520                  10  
133521                   5  
133522                   5  
133523                  10

### Preprocess wrestler data

In [12]:
# Read wrestler records

banzuke = pd.read_csv("data/banzuke.csv")
banzuke.tail()

basho     id   rank      rikishi       heya  shusshin  birth_date  \
174180  2022.09  12738  Jk17e      Okuyama    Hakkaku  Hokkaido  2003-06-03   
174181  2022.09  12777  Jk17w  Sachinofuji  Isegahama   Saitama  2006-03-06   
174182  2022.09    677  Jk18e  Itakozakura  Shikihide   Ibaraki  1978-10-02   
174183  2022.09   2905  Jk18w   Higohikari       Kise  Kumamoto  1988-01-27   
174184  2022.09  12546  Jk19e     Hamasaki      Otake   Fukuoka  2000-09-30   

        height  weight  prev  prev_w  prev_l  
174180   179.0   136.0    Mz     2.0     1.0  
174181   168.0    79.0    Mz     1.0     2.0  
174182   178.0   139.0  Jk2e     0.0     2.0  
174183   172.0   113.0  Jk8w     0.0     7.0  
174184   177.0   135.0    Mz     0.0     3.0

In [13]:
# Get several ids of famous sumo wrestlers

hakuho_id = banzuke[banzuke.rikishi == "Hakuho"].id.iloc[0]
print("Hakuho", hakuho_id)

kotooshu_id = banzuke[banzuke.rikishi == "Kotooshu"].id.iloc[0]
print("Kotooshu", kotooshu_id)

enho_id = banzuke[banzuke.rikishi == "Enho"].id.iloc[0]
print("Enho", enho_id)

Hakuho 1123
Kotooshu 2830
Enho 12412


In [14]:
# Filter out Makuuchi division

banzuke["rank"] = banzuke["rank"].str.extract(makuuchi_regex, expand=True)
banzuke = banzuke.dropna(subset=["rank"])
banzuke = banzuke.reset_index(drop=True)

In [15]:
# Extract year and month

year_and_month = get_year_and_month_from_date(banzuke.basho)
banzuke = pd.concat([banzuke, year_and_month], axis=1)

In [16]:
# Add column `age`

banzuke.basho = pd.to_datetime(banzuke.basho.astype(str), format="%Y.%m")
banzuke.birth_date = pd.to_datetime(banzuke.birth_date)
banzuke["age"] = banzuke.basho - banzuke.birth_date
# convert to years
banzuke["age"] = banzuke.age / np.timedelta64(1, "Y")

In [17]:
# Add column body mass index

banzuke["bmi"] = 1.3 * banzuke.weight / np.power(banzuke.height / 100, 2.5)

In [18]:
# Drop and re-order columns

banzuke = banzuke.drop(columns=[
    "basho",
    "birth_date", 
    "rank",
    "rikishi",
    "heya", 
    "shusshin", 
    "prev",
])
banzuke.columns = [
    'id',
    'height',
    'weight',
    'previous_tournament_wins',
    'previous_tournament_losses',
    'year',
    'month',
    'age',
    'bmi',
]
banzuke = banzuke[[
    'year',
    'month',
    'id',
    'height',
    'weight',
    'bmi',
    'age',
    'previous_tournament_wins',
    'previous_tournament_losses',
]]

In [19]:
# Fix data types

banzuke["id"] = banzuke["id"].astype(int)
banzuke.previous_tournament_wins = banzuke.previous_tournament_wins.astype(int)
banzuke.previous_tournament_losses = banzuke.previous_tournament_losses.astype(int)

In [20]:
# Print the clean wrestler data
banzuke.tail()

year  month     id  height  weight        bmi        age  \
9581  2022      9  12292   185.0   173.0  48.312569  28.942415   
9582  2022      9  11868   169.0   114.0  39.914569  27.622744   
9583  2022      9  12113   182.0   191.0  55.564607  31.099886   
9584  2022      9  12406   187.0   183.0  49.749695  28.353765   
9585  2022      9  12314   177.0   129.0  40.234619  22.365962   

      previous_tournament_wins  previous_tournament_losses  
9581                         8                           7  
9582                         6                           9  
9583                         5                           8  
9584                         9                           6  
9585                        10                           5

### Merge data

In [76]:
# Merge tournament and wrestler data

# Add wrestler 1 data
sumo = pd.merge(
    left = results,
    right = banzuke.add_suffix("1"),
    how = "left",
    left_on = ["year", "month", "id1"],
    right_on = ["year1", "month1", "id1"],
)
sumo = sumo.drop(columns=["year1", "month1",])

# Add wrestler 2 data
sumo = pd.merge(
    left = sumo,
    right = banzuke.add_suffix("2"),
    how = "left",
    left_on = ["year", "month", "id2"],
    right_on = ["year2", "month2", "id2"],
)
sumo = sumo.drop(columns=["year2", "month2",])

In [77]:
# Reorder columns

wrestler1_columns = sorted(list(sumo.columns[sumo.columns.str.contains("1")]))
wrestler2_columns = sorted(list(sumo.columns[sumo.columns.str.contains("2")]))
other_columns = ['year', 'month', 'day']

sumo = sumo[other_columns + wrestler1_columns + wrestler2_columns]

In [78]:
# Final clean `sumo` data

sumo.tail(2)

year  month  day       age1       bmi1  bout_outcome1  height1    id1  \
133522  2022      9   15  30.823357  48.122448              0    184.0  12130   
133523  2022      9   15  26.073088  58.721795              1    175.0  12191   

        previous_tournament_losses1  previous_tournament_wins1  ...  \
133522                            5                         10  ...   
133523                            4                         11  ...   

             bmi2  bout_outcome2  height2    id2  previous_tournament_losses2  \
133522  58.721795              1    175.0  12191                            4   
133523  48.122448              0    184.0  12130                            5   

        previous_tournament_wins2  rank2  tournament_losses2  \
133522                         11     20                   5   
133523                         10     20                  10   

        tournament_wins2  weight2  
133522                 9    183.0  
133523                 4    170.0  

[2 rows x 25 columns]

In [79]:
# Check for missing data
print(sumo.isna().sum())

year                           0
month                          0
day                            0
age1                           0
bmi1                           0
bout_outcome1                  0
height1                        0
id1                            0
previous_tournament_losses1    0
previous_tournament_wins1      0
rank1                          0
tournament_losses1             0
tournament_wins1               0
weight1                        0
age2                           0
bmi2                           0
bout_outcome2                  0
height2                        0
id2                            0
previous_tournament_losses2    0
previous_tournament_wins2      0
rank2                          0
tournament_losses2             0
tournament_wins2               0
weight2                        0
dtype: int64


In [80]:
# Verify data types
print(sumo.dtypes)

year                             int32
month                            int32
day                              int64
age1                           float64
bmi1                           float64
bout_outcome1                    int64
height1                        float64
id1                              int64
previous_tournament_losses1      int32
previous_tournament_wins1        int32
rank1                            int64
tournament_losses1               int64
tournament_wins1                 int64
weight1                        float64
age2                           float64
bmi2                           float64
bout_outcome2                    int64
height2                        float64
id2                              int64
previous_tournament_losses2      int32
previous_tournament_wins2        int32
rank2                            int64
tournament_losses2               int64
tournament_wins2                 int64
weight2                        float64
dtype: object


## Engineer new features

### Career wins

`career_wins1`
- Общ брой спечелени двубой до момента на конкретната среща.
- Total number of career wins in the top makuuchi division for this rikishi.
- The values are first calculated as the cumulative sum of the `bout_outcomes` for this rikishi.
- Then, they are shifted 1 row down to make them valid as "before the bout" statistics.
- Finally, the NA value generated in the first row after shifting is replaced with 0 as the rikishi had no wins at the start of his career. 

`total_bouts1`
- Общо изиграни двубой в Макуучи.
- Total number of played matches (bouts) by the rikishi in Makuuchi.

`career_win_percentage1`
- Win percentage.
- Процент спечелени двубой по време на цялата кариера.
- The percentage for the first 10 bouts is substituted for their average to accomodate for the low number of matches.

In [81]:
# Wrestler 1

dummy_sumo = pd.DataFrame()

for _, group_data in sumo.groupby("id1"):
    group_data["career_wins1"] = group_data["bout_outcome1"].cumsum()
    group_data.loc[:, "career_wins1"] = group_data.loc[:, "career_wins1"].shift(periods=1).fillna(0)

    group_data["career_bouts1"] = np.arange(0, len(group_data))

    group_data["career_win_percentage1"] = group_data["career_wins1"] / group_data["career_bouts1"]
    first_10_bouts = group_data["career_win_percentage1"].iloc[:10].mean()
    group_data.loc[group_data["career_bouts1"] < 10, "career_win_percentage1"] = first_10_bouts
    
    dummy_sumo = pd.concat([dummy_sumo, group_data], axis=0)

sumo = dummy_sumo.sort_index()

In [82]:
# Wrestler 2

dummy_sumo = pd.DataFrame()

for _, group_data in sumo.groupby("id2"):
    group_data["career_wins2"] = group_data["bout_outcome2"].cumsum()
    group_data.loc[:, "career_wins2"] = group_data.loc[:, "career_wins2"].shift(periods=1).fillna(0)

    group_data["career_bouts2"] = np.arange(0, len(group_data))

    group_data["career_win_percentage2"] = group_data["career_wins2"] / group_data["career_bouts2"]
    first_10_bouts = group_data["career_win_percentage2"].iloc[:10].mean()
    group_data.loc[group_data["career_bouts2"] < 10, "career_win_percentage2"] = first_10_bouts
    
    dummy_sumo = pd.concat([dummy_sumo, group_data], axis=0)

sumo = dummy_sumo.sort_index()

In [83]:
# Tests
print("\n Hakuho id=1123")
print(sumo[sumo.id1 == 1123][["career_wins1", "career_bouts1", "career_win_percentage1"]].tail())

print("\n Kotooshu id=2830")
print(sumo[sumo.id1 == 2830][["career_wins1", "career_bouts1", "career_win_percentage1"]].tail())

print("\n Enho id=12412")
print(sumo[sumo.id1 == 12412][["career_wins1", "career_bouts1", "career_win_percentage1"]].tail())


 Hakuho id=1123
        career_wins1  career_bouts1  career_win_percentage1
129318        1093.0           1296                0.843364
129356        1094.0           1297                0.843485
129394        1095.0           1298                0.843606
129432        1096.0           1299                0.843726
129470        1097.0           1300                0.843846

 Kotooshu id=2830
        career_wins1  career_bouts1  career_win_percentage1
104202         466.0            784                0.594388
104243         466.0            785                0.593631
104291         466.0            786                0.592875
104331         466.0            787                0.592122
104364         466.0            788                0.591371

 Enho id=12412
        career_wins1  career_bouts1  career_win_percentage1
127087          59.0            129                0.457364
127117          59.0            130                0.453846
127155          59.0            131             

### Encounter wins

#### Same-opponents match identifier
First, a new variable containing the encoded identifier of the pairs of wrestlers. It will be used to group the data for each pair of opponents.

`encounter_id`
- Unique identifier for each wrestler-opponent couple.
- The couples are counted twice - once from the prospective of each wrestler.

In [84]:
# Encode unique encounters
encounters = sumo.id1.astype(str) + ", " + sumo.id2.astype(str)
encounter_ids, _ = pd.factorize(encounters)
sumo["encounter_id1"] = encounter_ids

In [85]:
# Test
assert sumo.encounter_id1.nunique() == encounters.nunique()

In [86]:
# Test - single id for Hakuho (1123) and Kotooshu (2830) matches
assert sumo.loc[(sumo.id1 == 1123) & (sumo.id2 == 2830), "encounter_id1"].nunique() == 1

#### Select weighted moving average function

Moving average = rolling mean

Choosing between several types of weighted moving averages:
- with custom coef, considers only a certain number of previous records finite subset size.
- exponential coef, considers all previous records, pandas-builin method.

Custom one is better as it takes only certain number (4) of the most recent bouts and disregards the older ones. In contrast, EWM uses all bouts.

In [87]:
# Custom weighted moving average function
weights = np.array([0.05, 0.12, 0.25, 0.50])
def weighted_mean(subset):
    w = weights[len(weights) - len(subset):]
    return np.sum(w * subset) / w.sum()

In [88]:
# Compare methods
dummy_data = sumo.groupby("encounter_id1").get_group(12131)

bout_outcome1 = dummy_data.bout_outcome1
custom_mean = dummy_data.bout_outcome1.rolling(window=4, min_periods=1, closed="left").apply(weighted_mean)
ewm = dummy_data.bout_outcome1.ewm(span=4).mean().shift(1)

pd.DataFrame({
    "bout_outcome1": bout_outcome1,
    "custom": custom_mean,
    "ewm": ewm,
})

bout_outcome1    custom       ewm
74297              1       NaN       NaN
74535              0  1.000000  1.000000
75823              1  0.333333  0.375000
77375              0  0.712644  0.693878
82117              0  0.326087  0.375000
85069              1  0.130435  0.212353
85905              1  0.597826  0.542830
86195              1  0.815217  0.730965
87213              1  0.945652  0.840417
88647              0  1.000000  0.904900

The custom moving average gives 1 on the final row (88647). 

#### Generate variables

`encounter_bouts1`
- Общо изиграни двубой със същия опонент.
- Total number of played matches (bouts) with the same opponent.

`encounter_wins`
- Общ брой спечелени двубой с конкретния опонент до момента.
- Total number of wins vs the same oponent.
- Same calculation method as in `carrer_wins`

`encounter_win_percentage1`
- Процент спечелени двубой със същия опонент.
- Win percentage.
- The percentage for the first time the two opponents met is set to 0.5.

`encounter_win_rolling1`
- The weighted moving average (rolling mean) of the last 4 fights vs the same opponent.

In [90]:
# Generate encounter features

dummy_sumo = pd.DataFrame()

for _, group_data in sumo.groupby("encounter_id1"):
    group_data["encounter_bouts"] = np.arange(0, len(group_data))
    
    for i in ["1", "2"]:
        group_data[f"encounter_wins{i}"] = group_data[f"bout_outcome{i}"].cumsum().shift(1).fillna(0) 
        group_data[f"encounter_win_percentage{i}"] = group_data[f"encounter_wins{i}"] / group_data[f"encounter_bouts"]
        group_data[f"encounter_win_percentage{i}"] = group_data[f"encounter_win_percentage{i}"].fillna(0.5)
        group_data[f"encounter_win_rolling{i}"] = group_data[f"bout_outcome{i}"].ewm(span=4).mean().shift(1)
        group_data[f"encounter_win_rolling{i}"] = group_data[f"encounter_win_rolling{i}"].fillna(0.5)
    
    dummy_sumo = pd.concat([dummy_sumo, group_data], axis=0)

sumo = dummy_sumo.sort_index()

In [91]:
# Test with encounter_id=12131

sumo[sumo.encounter_id1 == 12131][[
    "bout_outcome1",
    "bout_outcome2",
    "encounter_bouts",
    "encounter_wins1",
    "encounter_win_percentage1",
    "encounter_win_rolling1",
    "encounter_wins2",
    "encounter_win_percentage2",
    "encounter_win_rolling2",
]]

bout_outcome1  bout_outcome2  encounter_bouts  encounter_wins1  \
74297              1              0                0              0.0   
74535              0              1                1              1.0   
75823              1              0                2              1.0   
77375              0              1                3              2.0   
82117              0              1                4              2.0   
85069              1              0                5              2.0   
85905              1              0                6              3.0   
86195              1              0                7              4.0   
87213              1              0                8              5.0   
88647              0              1                9              6.0   

       encounter_win_percentage1  encounter_win_rolling1  encounter_wins2  \
74297                   0.500000                0.500000              0.0   
74535                   1.000000                1.000000              0.0   
75823                   0.500000                0.375000              1.0   
77375                   0.666667                0.693878              1.0   
82117                   0.500000                0.375000              2.0   
85069                   0.400000                0.212353              3.0   
85905                   0.500000                0.542830              3.0   
86195                   0.571429                0.730965              3.0   
87213                   0.625000                0.840417              3.0   
88647                   0.666667                0.904900              3.0   

       encounter_win_percentage2  encounter_win_rolling2  
74297                   0.500000                0.500000  
74535                   0.000000                0.000000  
75823                   0.500000                0.625000  
77375                   0.333333                0.306122  
82117                   0.500000                0.625000  
85069                   0.600000                0.787647  
85905                   0.500000                0.457170  
86195                   0.428571                0.269035  
87213                   0.375000                0.159583  
88647                   0.333333                0.095100

### Tournament wins

Same idea as above. This time group by wrestler and tournament.

In [92]:
# Tournament wins

for i in ["1", "2"]:
    dummy_sumo = pd.DataFrame()
    
    for _, group_data in sumo.groupby(["year", "month", f"id{i}"]):
        group_data[f"tournament_wins{i}"] = group_data[f"bout_outcome{i}"].cumsum().shift(1).fillna(0) 
        group_data[f"tournament_bouts{i}"] = np.arange(0, len(group_data))
        group_data[f"tournament_win_percentage{i}"] = group_data[f"tournament_wins{i}"] / group_data[f"tournament_bouts{i}"]
        group_data[f"tournament_win_percentage{i}"] = group_data[f"tournament_win_percentage{i}"].fillna(0.5)

        group_data[f"tournament_win_rolling{i}"] = group_data[f"bout_outcome{i}"].ewm(span=4).mean().shift(1)
        group_data[f"tournament_win_rolling{i}"] = group_data[f"tournament_win_rolling{i}"].fillna(0.5)

        dummy_sumo = pd.concat([dummy_sumo, group_data], axis=0)


    sumo = dummy_sumo.sort_index()

In [94]:
# Test: Jan 2010 tournament, Kotooshu id1=2830

sumo[(sumo.year == 2010) & (sumo.month == 1) & (sumo.id1 == 2830)][[
    "bout_outcome1",
    "bout_outcome2",
    "tournament_wins1",
    "tournament_bouts1",
    "tournament_win_percentage1",
    "tournament_win_rolling1",
    "tournament_wins2",
    "tournament_bouts2",
    "tournament_win_percentage2",
    "tournament_win_rolling2",
]]

bout_outcome1  bout_outcome2  tournament_wins1  tournament_bouts1  \
89833              1              0               0.0                  0   
89869              1              0               1.0                  1   
89912              1              0               2.0                  2   
89957              0              1               3.0                  3   
89998              1              0               3.0                  4   
90033              1              0               4.0                  5   
90075              1              0               5.0                  6   
90117              1              0               6.0                  7   
90158              0              1               7.0                  8   
90196              0              1               7.0                  9   
90237              1              0               7.0                 10   
90275              0              1               8.0                 11   
90315              0              1               8.0                 12   
90353              0              1               8.0                 13   
90391              1              0               8.0                 14   

       tournament_win_percentage1  tournament_win_rolling1  tournament_wins2  \
89833                    0.500000                 0.500000               0.0   
89869                    1.000000                 1.000000               1.0   
89912                    1.000000                 1.000000               1.0   
89957                    1.000000                 1.000000               1.0   
89998                    0.750000                 0.540441               1.0   
90033                    0.800000                 0.739764               5.0   
90075                    0.833333                 0.848953               4.0   
90117                    0.857143                 0.911112               2.0   
90158                    0.875000                 0.947274               7.0   
90196                    0.777778                 0.564507               4.0   
90237                    0.700000                 0.337331               6.0   
90275                    0.727273                 0.603364               4.0   
90315                    0.666667                 0.361492              11.0   
90353                    0.615385                 0.216706              10.0   
90391                    0.571429                 0.129956              10.0   

       tournament_bouts2  tournament_win_percentage2  tournament_win_rolling2  
89833                  0                    0.500000                 0.500000  
89869                  1                    1.000000                 1.000000  
89912                  2                    0.500000                 0.375000  
89957                  3                    0.333333                 0.306122  
89998                  4                    0.250000                 0.459559  
90033                  5                    1.000000                 1.000000  
90075                  6                    0.666667                 0.328679  
90117                  7                    0.285714                 0.085333  
90158                  8                    0.875000                 0.981019  
90196                  9                    0.444444                 0.823402  
90237                 10                    0.600000                 0.893720  
90275                 11                    0.363636                 0.149532  
90315                 12                    0.916667                 0.988778  
90353                 13                    0.769231                 0.340476  
90391                 14                    0.714286                 0.526708

### Differences between opponents

Take the deltas (differences) between some statistics of the two opponents.

Is wrestler 1 ____ than his opponent:
- `age1` - much older / younger
- `bmi1` - much fitter
- `height1` - much higher
- `weight1` - much heavier
- `previous_tournament_win_percentage1` - had more success during the previous tournament 
- `previous_tournament_bouts1` - include to cover for high / low win percentage caused by low number of played bouts (e.g. only few bouts played, most were wins, and then the wrestler got injured and withdraw from the tournament)
- `rank1` - higher / lower rank
- `career_bouts1` - much more experienced (more matches)
- `career_win_percentage1` - much more successful during its carrrer
- `tournament_win_percentage1` - overall performing better during the tournament
- `tournament_win_rolling1` - in better shape during the latest matches

In [100]:
# Calculate previous tournament statistics
# TODO: move to Prepare data section

sumo["previous_tournament_bouts1"] = sumo["previous_tournament_wins1"] + sumo["previous_tournament_losses1"]
sumo["previous_tournament_bouts2"] = sumo["previous_tournament_wins2"] + sumo["previous_tournament_losses2"]

sumo["previous_tournament_win_percentage1"] = sumo["previous_tournament_wins1"] / sumo["previous_tournament_bouts1"]
sumo["previous_tournament_win_percentage2"] = sumo["previous_tournament_wins2"] / sumo["previous_tournament_bouts2"]

In [101]:
delta_columns = pd.Series([
    'age',
    'bmi',
    'height',
    'weight',
    'previous_tournament_win_percentage',
    'previous_tournament_bouts',
    'rank',
    'career_bouts',
    'career_win_percentage',
    'tournament_win_percentage',
    'tournament_win_rolling',
])

In [110]:
for column in delta_columns:
    sumo[column + "_delta"] = sumo[column + "1"] - sumo[column + "2"]

### Save cleaned data

In [117]:
# sumo = sumo.dropna()

In [118]:
# sumo.to_csv("cleaned_sumo.csv")

## Explore data

In [53]:
# TODO

## Model data

### Scale

1. Separate the output (label) variable - `bout_outcome1`.
1. Select a subset of predictor (attribute) columns.

In [119]:
bout_outcomes = sumo.bout_outcome1
sumo_attributes = sumo[[
    # Wrestler 1 features
    ## Characteristics
    'age1',
    'height1',
    'weight1',
    'bmi1',
    'rank1',
    
    ## Performance
    'career_bouts1',
    'career_win_percentage1',
  
    'tournament_win_percentage1',
    'tournament_win_rolling1',
    
    'previous_tournament_win_percentage1',
    'previous_tournament_bouts1',
    
    # Delta features
    ## Characteristics
    'age_delta',
    'bmi_delta',
    'height_delta',
    'weight_delta',
    'rank_delta',

    ## Performance
    'career_bouts_delta',
    'career_win_percentage_delta',
  
    'previous_tournament_win_percentage_delta',
    'previous_tournament_bouts_delta',
    
    'tournament_win_percentage_delta',
    'tournament_win_rolling_delta',
]]

In [120]:
sumo_attributes.isna().sum()

age1                                        0
height1                                     0
weight1                                     0
bmi1                                        0
rank1                                       0
career_bouts1                               0
career_win_percentage1                      0
tournament_win_percentage1                  0
tournament_win_rolling1                     0
previous_tournament_win_percentage1         0
previous_tournament_bouts1                  0
age_delta                                   0
bmi_delta                                   0
height_delta                                0
weight_delta                                0
rank_delta                                  0
career_bouts_delta                          0
career_win_percentage_delta                 0
previous_tournament_win_percentage_delta    0
previous_tournament_bouts_delta             0
tournament_win_percentage_delta             0
tournament_win_rolling_delta      

In [122]:
sumo_attributes

age1  height1  weight1       bmi1  rank1  career_bouts1  \
0       31.639253    184.0    130.0  36.799519      4              0   
1       27.685716    171.0    117.0  39.777633      5              0   
2       23.023060    187.0    126.0  34.253889      5              0   
3       27.384546    189.5    122.0  32.083382      6              0   
4       20.430262    188.0    143.0  38.360545      7              0   
...           ...      ...      ...        ...    ...            ...   
133519  27.737736    180.0    129.0  38.579074     19            199   
133520  28.906822    187.0    143.0  38.875445     12             74   
133521  29.684388    179.0    172.0  52.160197     20            602   
133522  30.823357    184.0    170.0  48.122448     20            589   
133523  26.073088    175.0    183.0  58.721795     20            452   

        career_win_percentage1  tournament_win_percentage1  \
0                     0.015625                    0.500000   
1                     0.677557                    0.500000   
2                     0.396693                    0.500000   
3                     0.810229                    0.500000   
4                     0.806526                    0.500000   
...                        ...                         ...   
133519                0.608040                    0.714286   
133520                0.567568                    0.642857   
133521                0.578073                    0.285714   
133522                0.531409                    0.285714   
133523                0.623894                    0.642857   

        tournament_win_rolling1  previous_tournament_win_percentage1  ...  \
0                      0.500000                             0.400000  ...   
1                      0.500000                             0.533333  ...   
2                      0.500000                             0.666667  ...   
3                      0.500000                             0.466667  ...   
4                      0.500000                             0.600000  ...   
...                         ...                                  ...  ...   
133519                 0.853041                             0.533333  ...   
133520                 0.815283                             0.400000  ...   
133521                 0.149541                             0.285714  ...   
133522                 0.727493                             0.666667  ...   
133523                 0.669556                             0.733333  ...   

        bmi_delta  height_delta  weight_delta  rank_delta  career_bouts_delta  \
0       -2.978114          13.0          13.0          -1                   0   
1        2.978114         -13.0         -13.0           1                   0   
2        2.170506          -2.5           4.0          -1                   0   
3       -2.170506           2.5          -4.0           1                   0   
4       -7.020303          -3.0         -33.0           1                   0   
...           ...           ...           ...         ...                 ...   
133519  -1.857996          -2.0         -10.0           6                -419   
133520 -13.284752           8.0         -29.0          -8                -528   
133521  13.284752          -8.0          29.0           8                 528   
133522 -10.599346           9.0         -13.0           0                 137   
133523  10.599346          -9.0          13.0           0                -137   

        career_win_percentage_delta  previous_tournament_win_percentage_delta  \
0                         -0.661932                                 -0.133333   
1                          0.661932                                  0.133333   
2                         -0.413536                                  0.200000   
3                          0.413536                                 -0.200000   
4                          0.543783                                 -0.066667   
...                